In [99]:
import numpy as np
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import random
import threading
import concurrent.futures

In [100]:
# Customize Chrome browser settings
chrome_options = Options()                                     
chrome_options.add_argument('--no-sandbox')                     #Them tuy chon de chay Chrome ko co sandbox, thuong duoc su dung de tranh cac van de ve quyen truy cap he thong
chrome_options.add_argument('--disable-notifications')          #Tat thong bao tu trinh duyet
chrome_options.add_argument('--disable-infobars')               #Tat thanh thong tin cua trinh duyet
#chrome_options.add_argument('--headless')

In [106]:
def getDataIn1Page(search_, page_, data_lock, data_crawled):
    base_link = 'https://www.lazada.vn/catalog/?page={page}&q={search}'
    link = base_link.format(page = page_, search = search_)
    # Access
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get("https://www.lazada.vn/trang-phuc-nu/?q=qu%E1%BA%A7n%20%C3%A1o&catalog_redirect_tag=true")
    driver.implicitly_wait(2)

    # Get link/title
    elems = driver.find_elements(By.CSS_SELECTOR , ".RfADt [href]")
    titles = [elem.text for elem in elems]
    links = [elem.get_attribute('href') for elem in elems]

    # Get type
    type_name = [elem.get_attribute("value") for elem in driver.find_elements(By.CSS_SELECTOR, ".search-box__input--O34g")]
    type_name = type_name * len(titles)

    # Get price sale
    elems_price_sale = driver.find_elements(By.CSS_SELECTOR , ".aBrP0")
    prices = [elem_price.text for elem_price in elems_price_sale]

    # Get discount, sold, preview
    discount_percent_list, sold_list, preview_list = [], [], []
    for i in range(1, len(titles)+1):
        try:
            discount_percent = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[4]/span".format(i))
            discount_percent_list.append(discount_percent.text)
        except NoSuchElementException:
            discount_percent_list.append(None)

        try:
            sold = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/span[1]/span[1]".format(i))
            sold_list.append(sold.text)
        except NoSuchElementException:
            sold_list.append(None)

        try:
            preview = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/div/span".format(i))
            preview_list.append(preview.text)
        except NoSuchElementException:
            preview_list.append(None)
    
    # Get Location
    elems_locations = driver.find_elements(By.CSS_SELECTOR , ".oa6ri ")
    locations = [elem.text for elem in elems_locations]

    # Cập nhật dữ liệu vào biến chung
    with data_lock:
        data_crawled['type'].extend(type_name)
        data_crawled['title'].extend(titles)
        data_crawled['links'].extend(links)
        data_crawled['price'].extend(prices)
        data_crawled['discount_percent'].extend(discount_percent_list)
        data_crawled['sold'].extend(sold_list)
        data_crawled['preview'].extend(preview_list)
        data_crawled['location'].extend(locations)


In [107]:
dataProducts = {
    "type": [],
    "title": [],
    "links": [],
    "price": [],
    "discount_percent": [],
    "sold": [],
    "preview": [],
    "location": []
}

In [103]:
def crawlDataWithMultiplesThread(product, num_pages, num_threads_per_loop):
    # Lock để đảm bảo thread-safe khi cập nhật dữ liệu
    data_lock = threading.Lock()
    
    # Lặp qua các trang và chạy các luồng
    for start_page in tqdm(range(1, num_pages + 1, num_threads_per_loop), desc="Crawling Data"):
        threads = []
        for page in range(start_page, start_page + num_threads_per_loop):
            thread = threading.Thread(target=getDataIn1Page, args=(product, page, data_lock, dataProducts))
            threads.append(thread)
            thread.start()

        # Đợi tất cả các luồng hoàn thành và thu thập dữ liệu đã crawl
        for thread in threads:
            thread.join()

        print(f"Crawling completed from pages {start_page} to {min(start_page + num_threads_per_loop - 1, num_pages)}")

    print("Crawling completed.")

    return dataProducts

In [96]:
dataProducts = crawlDataWithMultiplesThread("tủ+lạnh", 1, 1)

Crawling Data: 100%|██████████| 1/1 [01:41<00:00, 101.59s/it]

Crawling completed from pages 1 to 1
Crawling completed.


In [108]:
# Lock để đảm bảo thread-safe khi cập nhật dữ liệu
data_lock = threading.Lock()

In [109]:
getDataIn1Page(1,1,data_lock,dataProducts)

In [97]:
df = pd.DataFrame(dataProducts)

In [98]:
df

,type,title,links,price,discount_percent,sold,preview,location
0,tủ lạnh,[SALE SỐC 05.06 - 10.06: Chỉ 7xxxK] Tủ Lạnh Ng...,https://www.lazada.vn/products/sale-soc-0506-1...,12.790.000 ₫,25% Off,94 Đã bán,(37),Đồng Nai
1,tủ lạnh,"LZD 18L Máy Nén Ô Tô, Tủ Lạnh Ô Tô, Tủ Lạnh, T...",https://www.lazada.vn/products/lzd-18l-may-nen...,968.271 ₫,3% Off,None,None,Nước ngoài
2,tủ lạnh,Thanh lí tủ lạnh sanyou 90 lít giá rẻ LH 09667...,https://www.lazada.vn/products/thanh-li-tu-lan...,1.300.000 ₫,None,None,None,Hồ Chí Minh
3,tủ lạnh,Tủ lạnh tủ đông mini gia đình 5 cấp số tiết ki...,https://www.lazada.vn/products/tu-lanh-tu-dong...,2.521.857 ₫,56% Off,118 Đã bán,(17),Nam Định
4,tủ lạnh,Tủ lạnh mini 2 cánh 42L cao cấp với ngăn làm đ...,https://www.lazada.vn/products/tu-lanh-mini-2-...,2.327.119 ₫,55% Off,618 Đã bán,(96),Nam Định
5,tủ lạnh,Tủ lạnh mini 2 ngăn 42L có ngăn đá rời riêng b...,https://www.lazada.vn/products/tu-lanh-mini-2-...,2.316.237 ₫,52% Off,468 Đã bán,(69),Nam Định
6,tủ lạnh,[SALE SỐC 05.06 - 10.06: CHỈ 2xxxK] Tủ Lạnh 1 ...,https://www.lazada.vn/products/sale-soc-0506-1...,3.290.000 ₫,28% Off,None,(1),Đồng Nai
7,tủ lạnh,[SALE SỐC 05.06 - 10.06: CHỈ 2xxxK] Tủ Lạnh 1 ...,https://www.lazada.vn/products/sale-soc-0506-1...,3.290.000 ₫,28% Off,8 Đã bán,(1),Đồng Nai
8,tủ lạnh,[Trả góp 0%] Tủ lạnh Mini Aqua 90 Lít AQR-D99F...,https://www.lazada.vn/products/tra-gop-0-tu-la...,3.079.000 ₫,23% Off,5 Đã bán,(2),Hồ Chí Minh
9,tủ lạnh,[SALE SỐC 05.06 - 10.06: Chỉ 2xxxK] Tủ Lạnh 1 ...,https://www.lazada.vn/products/sale-soc-0506-1...,3.852.000 ₫,None,73 Đã bán,(16),Đồng Nai


In [111]:
# Hàm cuộn từ từ xuống dưới cùng của trang cho đến khi get element được thì dừng
def getStarItem(driver, step=200, wait_time=0.1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    current_position = 0
    
    while current_position < last_height:
        driver.execute_script(f"window.scrollBy(0, {step});")
        sleep(wait_time)
        current_position += step
        elems_stars = driver.find_elements(By.CSS_SELECTOR, ".percent")
        if not (elems_stars == []):
            break
    if not (elems_stars == []):
        return [elem.text for elem in elems_stars]
    else :
        return ['N/A'] * 5

In [112]:
def getDetailItems(link):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(link)
    driver.implicitly_wait(1)
    # get original price
    try:
        elems_original = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[10]/div/div/div/span[1]")
        original = elems_original.text
    except NoSuchElementException:
        original = None
    # get ship price
    try:
        elems_ship_price = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[1]/div/div/div[3]/div/div[1]/div/div[1]/div[2]")
        ship_price = elems_ship_price.text
    except NoSuchElementException:
        ship_price = None
    # get return_
    try:
        elems_return = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[3]/div/div[2]/div[2]/div/div/div/div")
        return_ = elems_return.text
    except NoSuchElementException:
        return_ = None
    # get positive review
    try:
        elems_positive_review = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[1]/div[2]")
        positive_review = elems_positive_review.text
    except NoSuchElementException:
        positive_review = None
    # get ship on time
    try:
        elems_ship_on_time = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[2]/div[2]")
        ship_on_time = elems_ship_on_time.text
    except NoSuchElementException:
        ship_on_time = None
    # get rate response
    try:
        elems_rate_response = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[3]/div[2]")
        rate_response = elems_rate_response.text
    except NoSuchElementException:
        rate_response = None
    # get stars
    stars = getStarItem(driver)
    star_5 = stars[0]
    star_4 = stars[1]
    star_3 = stars[2]
    star_2 = stars[3]
    star_1 = stars[4]
    # into a record
    # into a record
    data = {
        "original": original,
        "ship_price": ship_price,
        "return": return_,
        "positive_review": positive_review,
        "ship_on_time": ship_on_time,
        "rate_response": rate_response,
        "one_star": star_1,
        "two_star": star_2,
        "three_star": star_3,
        "four_star": star_4,
        "five_star": star_5
    }
    return data

In [113]:
detailProducts = {
    "original": [],
    "ship_price": [],
    "return": [],
    "positive_review": [],
    "ship_on_time": [],
    "rate_response": [],
    "one_star": [],
    "two_star": [],
    "three_star": [],
    "four_star": [],
    "five_star": []
}

In [114]:
# Sử dụng ThreadPoolExecutor để chạy đa luồng
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Sử dụng list comprehension để gọi hàm getDetailItems cho từng link trong df['links']
    results = list(tqdm(executor.map(getDetailItems, df['links']), total=len(df['links']), desc="Crawling product details"))

Crawling product details:  10%|█         | 4/40 [00:28<04:20,  7.23s/it]


WebDriverException: Message: disconnected: unable to send message to renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00C9B8E3+45827]
	(No symbol) [0x00C2DCC4]
	(No symbol) [0x00B2150F]
	(No symbol) [0x00B0C7EC]
	(No symbol) [0x00B0C776]
	(No symbol) [0x00B0B349]
	(No symbol) [0x00B0BD1D]
	(No symbol) [0x00B2479F]
	(No symbol) [0x00B24F62]
	(No symbol) [0x00B28298]
	(No symbol) [0x00B28310]
	(No symbol) [0x00B61ED6]
	(No symbol) [0x00B6216B]
	(No symbol) [0x00B9E0F2]
	(No symbol) [0x00B82E44]
	(No symbol) [0x00B9C034]
	(No symbol) [0x00B82B96]
	(No symbol) [0x00B56998]
	(No symbol) [0x00B5751D]
	GetHandleVerifier [0x00F54513+2899763]
	GetHandleVerifier [0x00FA793D+3240797]
	GetHandleVerifier [0x00D213B4+593364]
	GetHandleVerifier [0x00D282DC+621820]
	(No symbol) [0x00C370A4]
	(No symbol) [0x00C337A8]
	(No symbol) [0x00C33947]
	(No symbol) [0x00C259FE]
	BaseThreadInitThunk

In [328]:
# Create a progress bar to track the crawl process
for link in tqdm(df.links, desc="Crawling product details"):
    details = getDetailItems(link)
    for key in detailProducts.keys():
        detailProducts[key].append(details[key])

Crawling product details:  80%|████████  | 32/40 [16:14<03:14, 24.36s/it]

In [ ]:
# add columns to df
for key, value in detailProducts.items():
    df[key] = value

In [ ]:
# save to csv
df.to_csv("lazada.csv", index = True)
df.head()

,type,title,links,price,discount_percent,sold,preview,location,original,ship_price,return,positive_review,ship_on_time,rate_response,one_star,two_star,three_star,four_star,five_star
0,[Áo thun nữ],[Set Baggy Jeans Ống Suông Đai Nút Rách + Sơ m...,[https://www.lazada.vn/products/set-baggy-jean...,[149.000 ₫],[None],[5 Đã bán],[None],[Hồ Chí Minh],None,18.300 ₫,3 ngày trả hàng cho Nhà bán hàng,92%,99%,63%,0,0,0,0,0
1,[Áo thun nữ],"[Bộ Pijama, Đồ Bộ Mặc Nhà Nữ Dáng Cộc xinh xắn]",[https://www.lazada.vn/products/bo-pijama-do-b...,[15.971 ₫],[54% Off],[229 Đã bán],[(63)],[Hà Nội],59.000 ₫,32.700 ₫,3 ngày trả hàng cho Nhà bán hàng,95%,100%,100%,0,1,0,3,59
2,[Áo thun nữ],[Đồ Bộ Nữ Đi Chơi Chất Thun Mát Size M - dưới ...,[https://www.lazada.vn/products/do-bo-nu-di-ch...,[35.891 ₫],[64% Off],[3.0K Đã bán],[(1011)],[Hồ Chí Minh],100.000 ₫,16.500 ₫,3 ngày trả hàng cho Nhà bán hàng,97%,100%,100%,20,7,27,35,922
3,[Áo thun nữ],[Bộ mẫu hoạ tiết Sư Thích Minh Tuệ quần đùi ta...,[https://www.lazada.vn/products/bo-mau-hoa-tie...,[37.000 ₫],[8% Off],[78 Đã bán],[(9)],[Cần Thơ],45.000 ₫,28.700 ₫,3 ngày trả hàng cho Nhà bán hàng,96%,100%,60%,0,0,0,0,10
4,[Áo thun nữ],"[Đồ bộ nữ mặc nhà dễ thương , bộ ngủ pijama ng...",[https://www.lazada.vn/products/do-bo-nu-mac-n...,[25.000 ₫],[50% Off],[241 Đã bán],[(54)],[Hồ Chí Minh],75.999 ₫,16.500 ₫,3 ngày trả hàng cho Nhà bán hàng,91%,100%,57%,1,1,2,2,48
